In [147]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

### Load training data

In [161]:
A_train_targets = pd.read_parquet('./data/A/train_targets.parquet')
A_X_train_estimated = pd.read_parquet('./data/A/X_train_estimated.parquet')
A_X_train_observed = pd.read_parquet('./data/A/X_train_observed.parquet')
A_X_test = pd.read_parquet('./data/A/X_test_estimated.parquet')

B_train_targets = pd.read_parquet('./data/B/train_targets.parquet')
B_X_train_estimated = pd.read_parquet('./data/B/X_train_estimated.parquet')
B_X_train_observed = pd.read_parquet('./data/B/X_train_observed.parquet')
B_X_test = pd.read_parquet('./data/B/X_test_estimated.parquet')

C_train_targets = pd.read_parquet('./data/B/train_targets.parquet')
C_X_train_estimated = pd.read_parquet('./data/B/X_train_estimated.parquet')
C_X_train_observed = pd.read_parquet('./data/B/X_train_observed.parquet')
C_X_test = pd.read_parquet('./data/B/X_test_estimated.parquet')

X_train_estimated = {
    'A': A_X_train_estimated,
    'B': B_X_train_estimated,
    'C': C_X_train_estimated,
}
X_train_observed = {
    'A': A_X_train_observed,
    'B': B_X_train_observed,
    'C': C_X_train_observed,
}
train_targets = {
    'A': A_train_targets,
    'B': B_train_targets,
    'C': C_train_targets,
}
X_test = {
    'A': A_X_test,
    'B': B_X_test,
    'C': C_X_test,
}

X_train_estimated['A']


,date_calc,date_forecast,absolute_humidity_2m:gm3,air_density_2m:kgm3,ceiling_height_agl:m,clear_sky_energy_1h:J,clear_sky_rad:W,cloud_base_agl:m,dew_or_rime:idx,dew_point_2m:K,...,sun_azimuth:d,sun_elevation:d,super_cooled_liquid_water:kgm2,t_1000hPa:K,total_cloud_cover:p,visibility:m,wind_speed_10m:ms,wind_speed_u_10m:ms,wind_speed_v_10m:ms,wind_speed_w_1000hPa:ms
0,2022-10-27 07:00:18,2022-10-28 22:00:00,8.4,1.233,1425.099976,0.0,0.0,1211.699951,1.0,281.399994,...,340.799011,-38.466999,0.0,284.700012,100.000000,29429.699219,0.8,-0.4,0.7,-0.0
1,2022-10-27 07:00:18,2022-10-28 22:15:00,8.4,1.233,2085.899902,0.0,0.0,1560.500000,1.0,281.299988,...,345.451996,-38.955002,0.0,284.700012,100.000000,23604.099609,0.7,-0.4,0.7,-0.0
2,2022-10-27 07:00:18,2022-10-28 22:30:00,8.3,1.233,2746.600098,0.0,0.0,1909.400024,1.0,281.200012,...,350.153015,-39.310001,0.0,284.700012,100.000000,17799.800781,0.7,-0.3,0.6,-0.0
3,2022-10-27 07:00:18,2022-10-28 22:45:00,8.3,1.233,3407.399902,0.0,0.0,2258.199951,1.0,281.200012,...,354.886993,-39.528999,0.0,284.600006,100.000000,12016.500000,0.6,-0.2,0.6,-0.0
4,2022-10-27 07:00:18,2022-10-28 23:00:00,8.2,1.233,4068.199951,0.0,0.0,2607.000000,1.0,281.100006,...,359.638000,-39.609001,0.0,284.600006,100.000000,6254.399902,0.6,-0.1,0.6,-0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17571,2023-04-29 07:00:05,2023-04-30 22:45:00,4.5,1.281,1173.900024,0.0,0.0,536.500000,0.0,272.299988,...,352.920013,-11.731000,0.1,274.200012,99.900002,11629.299805,3.9,2.9,2.5,-0.0
17572,2023-04-29 07:00:05,2023-04-30 23:00:00,4.5,1.281,1054.199951,0.0,0.0,542.400024,0.0,272.200012,...,356.634003,-11.884000,0.1,274.200012,100.000000,9923.200195,3.7,2.8,2.4,-0.0
17573,2023-04-29 07:00:05,2023-04-30 23:15:00,4.5,1.281,1435.800049,0.0,0.0,531.799988,0.0,272.299988,...,0.352000,-11.928000,0.1,274.200012,99.900002,11230.799805,3.7,2.7,2.5,-0.0
17574,2023-04-29 07:00:05,2023-04-30 23:30:00,4.5,1.281,1817.400024,0.0,0.0,521.200012,0.0,272.299988,...,4.070000,-11.864000,0.1,274.200012,99.599998,12526.099609,3.8,2.7,2.6,-0.0


### Prepare data
result is a single dictionary `train_set_merged` with the keys `A`, `B` and `C`, for each corresponding location. one new feature is added: `location`, and the prediction-target is added as a columns as well: `y`

In [149]:
# Marking each row with its location origin
for location in X_train_estimated.keys():
    X_train_estimated[location]['location'] = location
    X_train_observed[location]['location'] = location
    X_test[location]['location'] = location
    X_train_estimated[location]['location'] = X_train_estimated[location]['location'].astype('category')
    X_train_observed[location]['location'] = X_train_observed[location]['location'].astype('category')
    X_test[location]['location'] = X_test[location]['location'].astype('category')

# Making a test set for each location
train_set_merged = {}
for location in X_train_estimated.keys():
    # concatinating observed and estimated
    concatinated = pd.concat(
        [X_train_observed[location], X_train_estimated[location]]
    )
    
    # merging target value into the training set
    merged = pd.merge(
        concatinated,
        train_targets[location],
        left_on='date_forecast',
        right_on='time',
        how='inner'
    )

    # dropping dates
    merged = merged.drop(columns=['date_calc', 'date_forecast'])

    # dropping some other shit
    merged = merged.drop(columns=['elevation:m', 'snow_density:kgm3', 'snow_drift:idx'], axis=1)

    # filling bad samples
    imputer = SimpleImputer(strategy='mean')
    merged[['ceiling_height_agl:m', 'cloud_base_agl:m']] = imputer.fit_transform(merged[['ceiling_height_agl:m', 'cloud_base_agl:m']])

    # dropping rows with NaN-values
    merged = merged.dropna()

    train_set_merged[location] = merged



#### Split into test-set and train-set
Both having seperate prediction-set

In [150]:
split_date = '2022-10-27'

X_train_dict = {}
y_train_dict = {}
X_test_dict = {}
y_test_dict = {}

for location in train_set_merged.keys():
    set = train_set_merged[location]
    set['time'] = pd.to_datetime(train_set_merged[location]['time'])
    set.sort_values('time', inplace=True)

    train_set = set[set['time'] < split_date]
    test_set = set[set['time'] >= split_date]

    X_train = train_set.drop(columns=['pv_measurement'])
    y_train = train_set['pv_measurement']
    X_test = test_set.drop(columns=['pv_measurement'])
    y_test = test_set['pv_measurement']

    X_train_dict[location] = X_train
    y_train_dict[location] = y_train
    X_test_dict[location] = X_test
    y_test_dict[location] = y_test

    


#### truncate data to one set regardless of location

In [151]:
X_train_concatinated = pd.concat(X_train_dict.values())
y_train_concatinated = pd.concat(y_train_dict.values())
X_train_concatinated['location'] = X_train_concatinated['location'].astype('category')

In [152]:
## temporary: drop categorical location. it does not work how it is supposed to

X_train_concatinated = X_train_concatinated.drop(columns=['location'])

for location in X_test_dict.keys():
    X_train_dict[location] = X_train_dict[location].drop(columns=['location'])
    X_test_dict[location] = X_test_dict[location].drop(columns=['location'])

## Training model

In [157]:
# make copy without time
X_train_concatinated_timeless = X_train_concatinated.drop(columns=['time'])

X_train_timeless = X_train_dict.copy()
X_test_timeless = X_test_dict.copy()
for location in X_test_dict.keys():
    X_train_timeless[location] = X_train_dict[location].drop(columns=['time'])
    X_test_timeless[location] = X_test_dict[location].drop(columns=['time'])

In [159]:

xgb_model = xgb.XGBRegressor(enable_categorical=True)
xgb_model.fit(X_train_concatinated_timeless, y_train_concatinated)

# Make predictions on the test set
y_pred = xgb_model.predict(X_test_timeless['C'])
y_pred = np.maximum(y_pred, 0)
# Calculate MAE on the test set
mae = mean_absolute_error(y_test_dict['C'], y_pred)
mae

/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarn

69.74698412387109

## Use trained model to generate submisstion file
submission file: `data/sample_submission.csv`

#### prepping test data

/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/torstein/school-projects/maskinlæring/MLGroupProject/venv/lib/python3.10/site-packages/xgboost/data.py:359: FutureWarn